In [1]:
from helpers.data import get_dataloaders
from helpers.train import TrainingManager
from helpers.loss_accuracy import accuracy
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.nn import CrossEntropyLoss, MSELoss
import models.resnet

In [2]:
trainloader_cifar, testloader_cifar = get_dataloaders('cifar10', 128)
device = torch.device('cuda:7')

Files already downloaded and verified


In [3]:
model_to_copy = models.resnet.resnet110()
model_to_copy.load_state_dict(torch.load('pretrained_models/resnet110-1d1ed7c2_new.th'))
model_to_copy = model_to_copy.to(device).eval()

In [5]:
model = models.resnet.resnet56().to(device)

In [15]:
model_1 = nn.Sequential(model.pre_layer, model.layer1)
model_2 = nn.Sequential(model_1, model.layer2)
model_3 = nn.Sequential(model_2, model.layer3)
model_4 = model

In [7]:
class IterableWraper():
    def __init__(self, ds, fn):
        self.ds = ds
        self.fn = fn
    def _generator(self):
        for x in self.ds:
            yield self.fn(x)
    def __iter__(self):
        return self._generator()

In [8]:
model_to_copy_1 = nn.Sequential(model_to_copy.pre_layer, model_to_copy.layer1)
model_to_copy_2 = nn.Sequential(model_to_copy_1, model_to_copy.layer2)
model_to_copy_3 = nn.Sequential(model_to_copy_2, model_to_copy.layer3)
model_to_copy_4 = model_to_copy

In [40]:
def f(x):
#     return x
    x, _ = x
    x = x.to(device)
    with torch.no_grad():
        return x, model_to_copy_3(x)

In [41]:
trainloader, testloader = IterableWraper(trainloader_cifar, f), IterableWraper(testloader_cifar, f)

In [42]:
trial_name = f"resnet110_mimic_with_resnet56"

In [43]:
optimizer = optim.SGD(model.parameters(), lr=1e-2, 
                      momentum=0.5, nesterov=True)
def lrfn(from_, to_, cycle_):
    def lr_(step):
        step = step % (cycle_ * 2)
        if step >= cycle_:
            step = cycle_ - (step - cycle_)  
        return 10 ** (from_ + (step / cycle_) * (to_ - from_))
    return lr_
lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lrfn(-4,-1,200))

In [44]:
tm = TrainingManager(trial_name, load=False, is_trial=False)

In [45]:
tm.train(model_3, optimizer,
         trainloader, testloader,
         
         MSELoss(), MSELoss(),
         MSELoss(), MSELoss(),
         
#          CrossEntropyLoss(), CrossEntropyLoss(),
#          accuracy, accuracy,
         
         lr_scheduler=lr_scheduler, device=device, no_iterations=10000)

  0%|          | 0/10000 [00:00<?, ?it/s]

Start training trial: resnet110_mimic_with_resnet56 is_trial


{tr_loss: 2.85729, tr_acc: 2.85729, te_loss: 3.24548, te_acc: 3.24548}:  38%|███▊      | 3760/10000 [05:07<07:47, 13.36it/s]      Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f41d41105c0>>
Traceback (most recent call last):
  File "/home/idokessler/dl-env/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/home/idokessler/dl-env/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
{tr_loss: 2.84080, tr_acc: 2.84080, te_loss: 3.23162, te_acc: 3.23162}:  38%|███▊      | 3760/10000 [05:07<07:47, 13.36it/s]AssertionError: can only join a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__d